# testを読み込んでsubmission.csvを出力する


In [1]:
import os
import numpy as np
import torch
import pandas as pd

# 1) Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cuda


# 適切なモデル構造を最初に作って、その後に学習済みモデルを読み込む

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WaveformEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=(3, 5, 3), padding=1)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool3d((1, 10, 10))   # 空間特徴を維持！
        self.fc = nn.Linear(32 * 10 * 10, 1024)
        self.out = nn.Linear(1024, 70 * 70)

    def forward(self, x):  # [B, 5, 1000, 7]
        x = x.unsqueeze(1)  # [B,1,5,1000,7]
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)                   # [B,32,1,10,10]
        x = x.view(x.size(0), -1)          # [B, 32*10*10]
        x = F.relu(self.fc(x))
        x = self.out(x)
        return x.view(-1, 1, 70, 70)       # [B, 1, 70, 70]

# Load pretrained model
model = WaveformEncoder().to(device)
model.load_state_dict(torch.load("best_pinn_model.pth", map_location=device))
model.eval()




WaveformEncoder(
  (conv1): Conv3d(1, 16, kernel_size=(3, 5, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool): AdaptiveAvgPool3d(output_size=(1, 10, 10))
  (fc): Linear(in_features=3200, out_features=1024, bias=True)
  (out): Linear(in_features=1024, out_features=4900, bias=True)
)

# 学習済みモデルでテストデータに対して推論。その後csvデータを出力

In [ ]:
import numpy as np
import torch
import pandas as pd
import os
from tqdm import tqdm  # 追加

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_dir = './waveform-inversion/test'
test_files = sorted([os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith('.npy')])

batch_size = 8  # 調整可能

rows = []

# tqdmでプログレスバー表示
for i in tqdm(range(0, len(test_files), batch_size), desc="Inference Progress"):
    batch_files = test_files[i:i+batch_size]
    batch_arrs = [np.load(fp) for fp in batch_files]  # list of arrays (5,1000,70)
    batch_tensor = torch.from_numpy(np.stack(batch_arrs)).float().to(device)  # (B,5,1000,70)

    with torch.no_grad():
        outs = model(batch_tensor)  # (B,1,70,70)

    outs = outs.squeeze(1).cpu().numpy()  # (B,70,70)

    for b, fp in enumerate(batch_files):
        fid = os.path.splitext(os.path.basename(fp))[0]
        out = outs[b]
        for y in range(out.shape[0]):
            row = {'oid_ypos': f'{fid}_y_{y}'}
            x_vals = {f'x_{2*j+1}': out[y, xpos] for j, xpos in enumerate(range(0, 70, 2))}
            row.update(x_vals)
            rows.append(row)

df = pd.DataFrame(rows)
cols = ['oid_ypos'] + [f'x_{i}' for i in range(1, 70, 2)]
df = df[cols]

df.head()
# df.to_csv('submission.csv', index=False)


Inference Progress:   1%|▎                           | 100/8228 [00:13<13:03, 10.38it/s]